In [2]:
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

import pandas as pd
import torch


In [3]:
def smape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + y_pred)))

In [4]:
PICKLE_FILE_PATH = "/data/avirinchipur/EMI/datadicts/voicemails_PCL11_minmax_ans_avg_roba128_rangel3u14.pkl"
datadict = pickle.load(open(PICKLE_FILE_PATH, "rb"))
print (datadict.keys())

dict_keys(['train_data', 'val_data', 'test_data'])


In [5]:
train_data, val_data = datadict['train_data'], datadict['val_data']
print ("Num train examples: {}".format(len(train_data['labels'])))
print ("Num val examples: {}".format(len(val_data['labels'])))


Num train examples: 122
Num val examples: 19


In [6]:
train_labels = list(map(lambda x: x[0], train_data['labels']))
val_labels = list(map(lambda x: x[0], val_data['labels']))
print ("Train Label distribution")
print ("Min: {}, Max: {}, Avg: {}, Median: {}".format(min(train_labels), max(train_labels), sum(train_labels)/len(train_labels), sorted(train_labels)[len(train_labels)//2]))
print ("Val Label distribution")
print ("Min: {}, Max: {}, Avg: {}, Median: {}".format(min(val_labels), max(val_labels), sum(val_labels)/len(val_labels), sorted(val_labels)[len(val_labels)//2]))

Train Label distribution
Min: 1.224744871391589, Max: 1.8540872566577051, Avg: 1.3584739428492187, Median: 1.3092415949028486
Val Label distribution
Min: 1.224744871391589, Max: 1.8602191717778138, Avg: 1.3582451887013627, Median: 1.324565602916875


In [7]:
avg_train_label = np.array([sum(train_labels)/len(train_labels)]*len(val_labels))
median_train_label = np.array([sorted(train_labels)[len(train_labels)//2]]*len(val_labels))
val_labels = np.array(val_labels)

In [8]:
print (avg_train_label[0], median_train_label[0])

1.3584739428492187 1.3092415949028486


In [9]:
mse_baseline_avg = mean_squared_error(val_labels, avg_train_label)
mae_baseline_avg = mean_absolute_error(val_labels, avg_train_label)
smape_baseline_avg = smape(val_labels, avg_train_label)

mse_baseline_median = mean_squared_error(val_labels, median_train_label)
mae_baseline_avg = mean_absolute_error(val_labels, median_train_label)
smape_baseline_median = smape(val_labels, median_train_label)

In [10]:
print ("Baseline Avg") 
print ("MSE: {}, MAE: {}, SMAPE: {}".format(round(mse_baseline_avg, 4), round(mae_baseline_avg, 4), round(smape_baseline_avg, 4)))
print ("Baseline Median")
print ("MSE: {}, MAE: {}, SMAPE: {}".format(round(mse_baseline_median, 4), round(mae_baseline_avg, 4), round(smape_baseline_median, 4)))

Baseline Avg
MSE: 0.0234, MAE: 0.1003, SMAPE: 0.0371
Baseline Median
MSE: 0.0258, MAE: 0.1003, SMAPE: 0.0355


In [11]:
val_labels, avg_train_label, median_train_label = ((val_labels/2)**2 - (3/8))*55 + 11, ((avg_train_label/2)**2 - (3/8))*55 + 11, ((median_train_label/2)**2 - (3/8))*55 + 11

mse_baseline_avg = mean_squared_error(val_labels, avg_train_label)
mae_baseline_avg = mean_absolute_error(val_labels, avg_train_label)
smape_baseline_avg = smape(val_labels, avg_train_label)

mse_baseline_median = mean_squared_error(val_labels, median_train_label)
mae_baseline_avg = mean_absolute_error(val_labels, median_train_label)
smape_baseline_median = smape(val_labels, median_train_label)

In [12]:
print ("Un Anscombed")
print ("Baseline Avg") 
print ("MSE: {}, MAE: {}, SMAPE: {}".format(round(mse_baseline_avg, 4), round(mae_baseline_avg, 4), round(smape_baseline_avg, 4)))
print ("Baseline Median")
print ("MSE: {}, MAE: {}, SMAPE: {}".format(round(mse_baseline_median, 4), round(mae_baseline_avg, 4), round(smape_baseline_median, 4)))

Un Anscombed
Baseline Avg
MSE: 40.6621, MAE: 3.9186, SMAPE: 0.1162
Baseline Median
MSE: 45.0546, MAE: 3.9186, SMAPE: 0.1104


In [13]:
PREDS_FILE_PATH = "/data/avirinchipur/EMI/outputs/voicemails/PCL11_minmax_ans_avg_roba128/voicemails_PCL11_minmax_ans_avg_hparams/8f399596359e423c9d861c882bcc1f7d/preds.pkl"
preds_dict = pickle.load(open(PREDS_FILE_PATH, "rb"))
preds_dict.keys()

dict_keys(['train', 'val', 'test'])

In [14]:
len(preds_dict['train']['preds'])

1000

In [15]:
preds, target = preds_dict['train']['preds'][737], preds_dict['train']['target'][737]
# mask = target != 0
# # Calculate Pearson correlation
# pearsonr(preds[mask], target[mask])

In [16]:
preds, target = preds_dict['val']['preds'][737][0], preds_dict['val']['target'][737][0]
mask = target != 0
# Calculate Pearson correlation
print ('Pearson r: ', pearsonr(preds[mask], target[mask]))
print ('MSE: ', mean_squared_error(preds[mask], target[mask]))
print ('MAE: ', mean_absolute_error(preds[mask], target[mask]))
print ('SMAPE: ', smape(preds[mask].numpy(), target[mask].numpy()))

Pearson r:  (0.19300775252363028, 0.0024624545171336213)
MSE:  0.16199276
MAE:  0.25019816
SMAPE:  0.12055549


In [17]:
timestep_level_preds = {}
epoch=737
for batch_pred, batch_target in zip(preds_dict['val']['preds'][epoch], preds_dict['val']['target'][epoch]):
    batch_mask = ~(batch_target==0)
    for time_step in range(batch_pred.shape[1]):
        if time_step not in timestep_level_preds: timestep_level_preds[time_step] = {'preds': [], 'target': [], 'mask': []}
        timestep_level_preds[time_step]['preds'].extend(batch_pred[:, time_step])
        timestep_level_preds[time_step]['target'].extend(batch_target[:, time_step])
        timestep_level_preds[time_step]['mask'].extend(batch_mask[:, time_step])

In [18]:
timestep_level_metrics = {}
for i in timestep_level_preds:
    preds, target, mask = timestep_level_preds[i]['preds'], timestep_level_preds[i]['target'], timestep_level_preds[i]['mask']
    preds, target, mask = torch.tensor(preds), torch.tensor(target), torch.tensor(mask)
    preds, target = ((preds/2)**2 - (3/8))*55 + 11, ((target/2)**2 - (3/8))*55 + 11
    mse_timestep = torch.sum(torch.square(preds - target)*mask)/torch.sum(mask)
    mae_timestep = torch.sum(torch.abs(preds - target)*mask)/torch.sum(mask)
    smape_timestep = torch.sum(torch.abs(preds - target)/(torch.abs(preds) + torch.abs(target) + 1e-8)*mask)/torch.sum(mask)
    count_timestep = torch.sum(mask)
    timestep_level_metrics[i] = {'mse': mse_timestep.item(), 'mae': mae_timestep.item(), 'smape': smape_timestep.item(), 'count': count_timestep.item()}

In [19]:
pd.DataFrame(timestep_level_metrics).T

,mse,mae,smape,count
0,52.231110,4.722153,0.145023,19.0
1,106.585320,6.965294,0.264477,19.0
2,183.671432,8.787758,0.345185,19.0
3,212.117142,9.768369,0.385870,19.0
4,134.728790,6.620628,0.232378,19.0
5,159.753937,8.677685,0.344561,19.0
6,139.586975,7.271071,0.281410,19.0
7,171.384735,9.651164,0.401910,19.0
8,135.526276,8.970512,0.400401,17.0
9,111.603188,7.215124,0.287426,17.0


In [20]:
epoch_metrics = pickle.load(open("/data/avirinchipur/EMI/outputs/voicemails/PCL11_minmax_ans_avg_roba128/voicemails_PCL11_minmax_ans_avg_hparams/8f399596359e423c9d861c882bcc1f7d/epoch_metrics.pkl", "rb"))
print (epoch_metrics.keys())

dict_keys(['train', 'val', 'test'])


In [21]:
epoch_metrics['val']['pearsonr'][:15]

[tensor(0.0096),
 tensor(0.0197),
 tensor(-0.0256),
 tensor(0.0092),
 tensor(-0.0480),
 tensor(-0.0025),
 tensor(-0.0527),
 tensor(-0.0081),
 tensor(-0.0263),
 tensor(-0.0035),
 tensor(-0.0765),
 tensor(0.0044),
 tensor(-0.0135),
 tensor(0.0107),
 tensor(-0.0665)]

In [22]:
preds_dict['train']['preds'][0][0]

tensor([[ 0.4240,  0.1044, -0.1692,  ...,  0.1044,  0.1044,  0.1044],
        [-0.0439,  0.1044,  0.1044,  ...,  0.1044,  0.1044,  0.1044],
        [ 0.3634, -0.0163, -0.5928,  ...,  0.1044,  0.1044,  0.1044],
        ...,
        [-0.1335,  0.1044,  0.1044,  ...,  0.1044,  0.1044,  0.1044],
        [ 0.0063, -0.2970,  0.0112,  ...,  0.1044,  0.1044,  0.1044],
        [ 0.2837, -0.3592,  0.1485,  ...,  0.1044,  0.1044,  0.1044]])

In [23]:
preds_dict['train']['target'][0][0]

tensor([[1.3234, 1.3234, 1.3234,  ..., 0.0000, 0.0000, 0.0000],
        [1.2939, 1.2939, 1.2939,  ..., 0.0000, 0.0000, 0.0000],
        [1.8259, 1.8259, 1.8259,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [1.3462, 1.3462, 1.3462,  ..., 0.0000, 0.0000, 0.0000],
        [1.5181, 1.5181, 1.5181,  ..., 0.0000, 0.0000, 0.0000],
        [1.3873, 1.3873, 1.3873,  ..., 0.0000, 0.0000, 0.0000]])

In [24]:
preds_dict['val']['target'][737][0]

tensor([[1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378,
         1.2378, 1.2378, 1.2378, 1.2378, 0.0000, 0.0000],
        [1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354,
         1.2354, 1.2354, 1.2354, 1.2354, 0.0000, 0.0000],
        [1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674,
         1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674],
        [1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954,
         1.3954, 1.3954, 1.3954, 1.3954, 0.0000, 0.0000],
        [1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819,
         1.2819, 1.2819, 1.2819, 1.2819, 0.0000, 0.0000],
        [1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805,
         1.2805, 1.2805, 1.2805, 0.0000, 0.0000, 0.0000],
        [1.3344, 1.3344, 1.

In [25]:
 preds_dict['val']['preds'][737][0]

tensor([[1.2236, 1.2217, 0.4925, 0.4925, 1.2778, 1.2409, 1.2785, 0.4925, 0.4925,
         1.2187, 0.4925, 1.2503, 1.3347, 0.4925, 0.4925],
        [1.2347, 1.2983, 1.2759, 1.2387, 0.4925, 1.2679, 0.4925, 1.3043, 0.4925,
         0.4925, 0.4925, 0.4925, 0.4925, 0.4925, 0.4925],
        [1.2127, 1.2254, 1.2559, 1.2812, 1.2436, 0.4925, 1.2916, 1.2679, 0.4925,
         1.2723, 0.4925, 1.2706, 1.2240, 0.4925, 0.4925],
        [1.2501, 1.2658, 1.2319, 0.4925, 1.2838, 1.2723, 1.2615, 1.2663, 1.2741,
         1.3097, 1.3122, 1.2970, 1.3628, 1.3367, 1.4001],
        [1.2318, 0.4925, 1.4025, 1.3913, 1.3909, 1.4348, 1.4332, 0.4925, 1.4225,
         1.3980, 1.3712, 1.4412, 1.4167, 0.4925, 0.4925],
        [1.1954, 0.4925, 0.4925, 1.1926, 1.1871, 1.4229, 1.4598, 1.4872, 0.4925,
         0.4925, 0.4925, 1.3038, 1.2352, 0.4925, 0.4925],
        [1.2388, 1.2085, 1.2748, 0.4925, 1.2779, 1.2607, 1.2631, 0.4925, 1.3802,
         1.3342, 1.3193, 1.3216, 0.4925, 0.4925, 0.4925],
        [1.2921, 1.3011, 1.

In [26]:
new_preds = pickle.load(open("/data/avirinchipur/EMI/outputs/voicemails/PCL11_minmax_ans_avg_roba128/voicemails_PCL11_minmax_ans_avg_hparams/3fcbf630493e4de093db0de11b6e5c79/preds.pkl", "rb"))
print (new_preds.keys())

dict_keys(['train', 'val', 'test'])


In [27]:
new_preds['val']['preds'][737][0]

tensor([[1.2378, 1.2181, 1.2031, 1.1998, 1.2063, 1.1945, 1.2662, 1.3259, 1.3704,
         1.3203, 1.2972, 1.3380, 1.3991, 0.4929, 0.4929],
        [1.2663, 1.3347, 1.3158, 1.2937, 1.3017, 1.3132, 1.3163, 1.3546, 0.4929,
         0.4929, 0.4929, 0.4929, 0.4929, 0.4929, 0.4929],
        [1.1833, 1.2463, 1.2706, 1.2805, 1.2552, 1.2603, 1.3025, 1.2892, 1.2814,
         1.2918, 1.3089, 1.3204, 1.2801, 0.4929, 0.4929],
        [1.2210, 1.2649, 1.2233, 1.2027, 1.2559, 1.2566, 1.2398, 1.2667, 1.2773,
         1.3225, 1.3033, 1.2977, 1.3466, 1.3272, 1.4081],
        [1.2005, 1.2343, 1.3800, 1.3731, 1.3495, 1.3957, 1.3968, 1.4027, 1.4316,
         1.3799, 1.3727, 1.4237, 1.4047, 0.4929, 0.4929],
        [1.2610, 1.2024, 1.1572, 1.1394, 1.1262, 1.2430, 1.2913, 1.3432, 1.3868,
         1.4183, 1.4414, 1.3513, 1.2584, 0.4929, 0.4929],
        [1.2690, 1.2541, 1.3022, 1.3381, 1.3373, 1.3139, 1.3292, 1.3421, 1.3915,
         1.3625, 1.3448, 1.3505, 0.4929, 0.4929, 0.4929],
        [1.2282, 1.2722, 1.

In [28]:
new_preds['val']['target'][737][0]

tensor([[1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378,
         1.2378, 1.2378, 1.2378, 1.2378, 0.0000, 0.0000],
        [1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354,
         1.2354, 1.2354, 1.2354, 1.2354, 0.0000, 0.0000],
        [1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674,
         1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674],
        [1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954,
         1.3954, 1.3954, 1.3954, 1.3954, 0.0000, 0.0000],
        [1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819, 1.2819,
         1.2819, 1.2819, 1.2819, 1.2819, 0.0000, 0.0000],
        [1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805, 1.2805,
         1.2805, 1.2805, 1.2805, 0.0000, 0.0000, 0.0000],
        [1.3344, 1.3344, 1.

In [48]:
new_preds_0infill = pickle.load(open("/data/avirinchipur/EMI/outputs/voicemails/PCL11_minmax_ans_avg_roba128/voicemails_PCL11_minmax_ans_avg_hparams/234aef79072a4ae6aeff83c0f243118d/preds.pkl", "rb"))
print (new_preds_0infill['train'].keys())

dict_keys(['preds', 'target', 'mask', 'infill_mask'])


In [55]:
new_preds_0infill['val']['infill_mask'][737][0][:5]

tensor([[False, False,  True,  True, False, False, False,  True,  True, False,
          True, False, False, False, False],
        [False, False, False, False,  True, False,  True, False, False, False,
         False, False, False, False, False],
        [False, False, False, False, False,  True, False, False,  True, False,
          True, False, False, False, False],
        [False, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False],
        [False,  True, False, False, False, False, False,  True, False, False,
         False, False, False, False, False]])

In [53]:
new_preds_0infill['val']['target'][737][0][:5]

tensor([[1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378, 1.2378,
         1.2378, 1.2378, 1.2378, 1.2378, 0.0000, 0.0000],
        [1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 1.3555, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354, 1.2354,
         1.2354, 1.2354, 1.2354, 1.2354, 0.0000, 0.0000],
        [1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674,
         1.3674, 1.3674, 1.3674, 1.3674, 1.3674, 1.3674],
        [1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954, 1.3954,
         1.3954, 1.3954, 1.3954, 1.3954, 0.0000, 0.0000]])

In [54]:
new_preds_0infill['val']['preds'][737][0][:5]

tensor([[1.2378, 1.2181, 1.2031, 1.1998, 1.2063, 1.1945, 1.2662, 1.3259, 1.3704,
         1.3203, 1.2972, 1.3380, 1.3991, 0.4929, 0.4929],
        [1.2663, 1.3347, 1.3158, 1.2937, 1.3017, 1.3132, 1.3163, 1.3546, 0.4929,
         0.4929, 0.4929, 0.4929, 0.4929, 0.4929, 0.4929],
        [1.1833, 1.2463, 1.2706, 1.2805, 1.2552, 1.2603, 1.3025, 1.2892, 1.2814,
         1.2918, 1.3089, 1.3204, 1.2801, 0.4929, 0.4929],
        [1.2210, 1.2649, 1.2233, 1.2027, 1.2559, 1.2566, 1.2398, 1.2667, 1.2773,
         1.3225, 1.3033, 1.2977, 1.3466, 1.3272, 1.4081],
        [1.2005, 1.2343, 1.3800, 1.3731, 1.3495, 1.3957, 1.3968, 1.4027, 1.4316,
         1.3799, 1.3727, 1.4237, 1.4047, 0.4929, 0.4929]])

In [44]:
new_epoch_metrics_0infill = pickle.load(open("/data/avirinchipur/EMI/outputs/voicemails/PCL11_minmax_ans_avg_roba128/voicemails_PCL11_minmax_ans_avg_hparams/234aef79072a4ae6aeff83c0f243118d/epoch_metrics.pkl", "rb"))
print (new_epoch_metrics_0infill.keys())

dict_keys(['train', 'val', 'test'])


In [46]:
len(new_epoch_metrics_0infill['val']['pearsonr'])

1001